In [ ]:
import os.path
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from IPython.display import display
import ipywidgets as widgets

def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def doEvaV64(x, N, rho, N_bg, plot=True):
    # Lineare Regression
    y = N - N_bg
    slope, intercept, r_value, p_value, std_err = linregress(x, np.log(y))

    if plot:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

        ax1.grid()
        ax1.set_title("Lineare Skala")

        ax2.grid(True, which="both")
        ax2.semilogy()
        ax2.set_title("Halblogarithmische Skala")

        for ax in [ax1, ax2]:
            ax.plot(x, y, marker='x', markersize=10, ls='', label="Daten")
            ax.plot(x, np.exp((slope * x + intercept)), label="Fit")
            ax.set_xlabel("Absorberdicke [mm]")
            ax.set_ylabel("Pulszahl")
            #ax.set_ylim(100, 1200)
            ax.legend()

        plt.show()

    mu = - slope * 10  # 1/cm
    print("Abschwächungskoeffizient µ = {0:.3f} cm⁻¹".format(mu))
    print("Halbwertsdicke d12 = {0:.3f} cm".format(np.log(2) / mu))
    print("Massenschächungskoeffizient µ/ρ = {0:.3f} cm²/g".format(mu / rho))

def evaluate_all(data_dir):
    N_bg = np.load(os.path.join(data_dir, "DataV64Nbg.npy"))
    
    for name, rho in [("Alu", 2.70), ("Cu", 8.96)]:
        display(H1(name))
        
        data = np.load(os.path.join(data_dir, f"DataV64{name}.npy"))
        x = data[0] # mm
        N = data[1]

        doEvaV64(x, N, rho, N_bg)

    display(H1("Frage nach der Durchführung"))

    mu_m = 0.06 # cm² / g
    rho_pb = 11.35 # g / cm³
    mu_pb = mu_m * rho_pb
    print("Schwächungskoeffizient µ_pb = {0:.3f} cm⁻¹".format(mu_pb))
    print("Halbwertsdicke = {0:.3f} cm".format(np.log(2) / mu_pb)) # cm

# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    plt.close("all")
    
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V64", name_escaped)
    
    evaluate_all(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})